## Table of Contents

## Part 1 資料清理、特徵工程
* [Setup](#Setup)
* 特徵處理
|主檔|來源檔案|輸出檔案|資料角色|
|:-|:-|:-|:-:|
|[00_帳號登入資料](#01_帳號登入資料)|sessionActive.csv|target.parquet|Y: 是否流失|
|[01_帳號登入資料](#01_帳號登入資料)|sessionActive.csv|feature_01.parquet|X|
|[02_帳號創建資料](#02_帳號創建資料)|accountCreate.csv|feature_02.parquet|X|
|[03_遊戲時長資料](#03_遊戲時長資料)|sessionLength.csv|feature_03.parquet|X|
|[04_遊戲押注紀錄](#04_遊戲押注紀錄)|betRecords.csv|feature_04.parquet|X|
|[05_金幣進出紀錄](#05_金幣進出紀錄)|coinRecords.csv|feature_05.parquet|X|
|[06_轉帳交易紀錄](#06_轉帳交易紀錄)|txnRecords.csv|feature_06.parquet|X|
|[07_儲值紀錄](#07_儲值紀錄)|depositRecords.csv|feature_07.parquet|X|
|[08_儲值退費紀錄](#08_儲值退費紀錄)|refundRecords.csv|feature_08.parquet|X|


* [合併所有特徵](#合併所有特徵)


## Part 2 玩家流失模型建置
1. [資料處理](#1.-資料處理)
2. [模型使用](#2.-模型使用)

---
# Part1 資料清理、特徵工程
### Setup
[Top](#Table-of-Contents)

#### 匯入所需Python套件

In [1]:
#需自行安裝
# pip install -U imbalanced-lear
# pip install collinearity
# pip install reliability
# pip install xgboost
# 01 一般使用套件
import pandas as pd
import numpy as np
import datetime, os, re
from collections import Counter
from tqdm.notebook import tqdm as tqdm
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

# 02 畫圖套件
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
import plotly.express as px


# 03 資料處理套件
from imblearn.over_sampling import SMOTE, KMeansSMOTE, BorderlineSMOTE #需自行安裝pip install -U imbalanced-lear
import sklearn
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.impute import SimpleImputer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel, SelectKBest, mutual_info_classif
from collinearity import SelectNonCollinear #需自行安裝pip install collinearity
from reliability.Probability_plotting import Normal_probability_plot #需自行安裝pip install reliability
from reliability.Probability_plotting import Normal_Distribution


# 04 巨量資料處理套件
# import multiprocessing
# from dask import dataframe as dd

# 05 機器學習套件
import xgboost as xgb #需自行安裝pip install xgboost
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn import tree, preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RepeatedStratifiedKFold

# 06 jupyter 輸出顯示設定
from IPython.display import display, Markdown, Image #required for well-formatted output from Notebook cells
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)



# 07 現在目錄位址
os.chdir('D:\\摩可\\新語法\\Input_data\\202410_04')
base_dir = os.getcwd().replace('04_scripts', '')
base_dir

'D:\\摩可\\新語法\\Input_data\\202410_04'

#### 自定義Python函式

In [2]:
def parallel_processing(df, func):
    num_cores = multiprocessing.cpu_count() - 1 # leave one free to not freeze machine
    df_split = np.array_split(df, num_cores)
    
    pool = multiprocessing.Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    
    pool.close()
    pool.join()
    
    return df


def calculate_slope(df):
    copy = df.copy()
    
    # 初始化一個empty dataframe
    account_id_list = list(copy['玩家帳號'].unique())
    feature_list = list(copy.columns)
    feature_list.remove('玩家帳號')
    feature_list.remove('周次')
    slope = pd.DataFrame(index = account_id_list, columns = feature_list)
    slope.fillna(value = np.nan, inplace = True)
    
    # 逐account id計算斜率值
    for index, row in slope.iterrows():
        account = copy[copy['玩家帳號'] == index]
        try:
            for i in feature_list:
                slope.loc[index, i] = np.polyfit(account['周次'], account[i], 1)[0]
        except:
            slope.loc[index, i] = 0
    return slope


def standardize_slope(df):
    copy = df.copy()
    
    columns = list(copy.columns)
    columns.remove('玩家帳號')

    for c in columns:
        copy[c + '_abs'] = abs(copy[c])
        copy[c + '_rank'] = copy[c + '_abs'].rank(method = 'min', pct = True)
        copy[c + '_std'] = np.where(copy[c] < 0 , round(copy[c + '_rank'], 2) * (-1), round(copy[c + '_rank'], 2))
        del copy[c]
        del copy[c + '_abs']
        del copy[c + '_rank'] 
        copy.rename(columns = {c + '_std':c}, inplace = True)
        
    return copy

def write_feature_name(df, source):
    with open(r'C:\Users\twnfkl\Desktop\03_feature data\features.csv', mode='a', encoding='utf-8') as file:
        columns = list(df.columns)
        columns.remove('玩家帳號')
        for i in list(columns):
            file.write(f'{i},{source}\n')

#### 設定觀察期、表現期各別的起日、迄日

In [49]:
obs_start_date_wg = widgets.DatePicker(description = '觀察期起日：', disabled = False)
obs_start_date_wg.value = datetime.date(2024, 9, 24)
obs_start_date_wg

DatePicker(value=datetime.date(2024, 9, 24), description='觀察期起日：')

In [50]:
obs_end_date_wg = widgets.DatePicker(description = '觀察期迄日：', disabled = False)
obs_end_date_wg.value = datetime.date(2024, 10, 7)
obs_end_date = obs_end_date_wg.value  # 取得選擇的日期
obs_week_number = obs_end_date.isocalendar()[1]  # 計算是該年的第幾週
obs_end_date_wg
# obs_week_number

DatePicker(value=datetime.date(2024, 10, 7), description='觀察期迄日：')

In [51]:
perf_start_date_wg = widgets.DatePicker(description = '表現期起日：', disabled = False)
perf_start_date_wg.value = datetime.date(2024, 10, 22)
perf_start_date_wg

DatePicker(value=datetime.date(2024, 10, 22), description='表現期起日：')

In [52]:
perf_end_date_wg = widgets.DatePicker(description = '表現期迄日：', disabled = False)
perf_end_date_wg.value = datetime.date(2024, 11, 4)
perf_end_date_wg

DatePicker(value=datetime.date(2024, 11, 4), description='表現期迄日：')

In [53]:
obs_start_date = obs_start_date_wg.value
obs_end_date = obs_end_date_wg.value
perf_start_date = perf_start_date_wg.value
perf_end_date = perf_end_date_wg.value

---
### 00_帳號登入資料
##### sessionActive.csv-> 變數處理-> target.parquet
[Top](#Table-of-Contents)

#### 決定目標玩家，以及是否流失的二元註記

In [54]:
%%time
# Wall time: 11min 59s
    
# 01 讀取CSV資料
df_01 = pd.read_csv(base_dir + r'\02_raw data\01_sessionActive\sessionActive.csv',
                    dtype = {'transferUID': 'object', 'LoginTimes': 'int8'},
                    parse_dates = ['LoginDate'])

df_01.rename(
    inplace = True,
    columns = {
        'LoginDate':'登入日期',
        'transferUID':'玩家帳號',
        'LoginTimes':'登入次數'
    }    
)


# 02 特徵處理
## 篩選出於觀察期有登入紀錄的玩家
df_01_obs = df_01.copy()
df_01_obs = df_01_obs[df_01_obs['登入次數'] > 0]
df_01_obs = df_01_obs[(obs_start_date <= df_01_obs['登入日期'].dt.date) & (df_01_obs['登入日期'].dt.date <= obs_end_date)]
df_01_obs_uni = pd.DataFrame(df_01_obs.玩家帳號.unique(), columns=['玩家帳號'])

## 篩選出於表現期有登入紀錄的玩家
df_01_perf = df_01.copy()
df_01_perf = df_01_perf[(perf_start_date <= df_01_perf['登入日期'].dt.date) & (df_01_perf['登入日期'].dt.date <= perf_end_date)]
df_01_perf_uni = pd.DataFrame(df_01_perf.玩家帳號.unique(), columns=['玩家帳號'])

## 左聯集即為建模使用的目標玩家
target = pd.merge(df_01_obs_uni, df_01_perf_uni, on = '玩家帳號', how = 'left', indicator = True)
target['是否流失'] = target['_merge'].apply(lambda x: '是' if x == 'left_only' else '否')
# target = target.sample(n = 10000)
del target['_merge']


# 03 儲存特徵資料
# target.to_parquet(base_dir + r'\03_feature data\target.parquet', index = False)


# 04 呈現資料Shape
# target = pd.read_parquet(base_dir + r'\03_feature data\target.parquet')

print(f'資料筆數: {target.shape[0]:,}\n欄位數量: {target.shape[1]:}')
print(f'流失比例: {Counter(target["是否流失"]).get("是") / target.shape[0]:.2%}')

target.head()

資料筆數: 75,707
欄位數量: 2
流失比例: 100.00%
CPU times: total: 875 ms
Wall time: 1.27 s


,玩家帳號,是否流失
0,12437148,是
1,12363509,是
2,11802253,是
3,12419390,是
4,12432207,是


---
### 01_帳號登入資料
##### sessionActive.csv-> 特徵處理-> feature_01.parquet
[Top](#Table-of-Contents)

In [55]:
%%time
# Wall time: 38.2 s
    
# 01 讀取CSV資料
df_01 = pd.read_csv(base_dir + r'\02_raw data\01_sessionActive\sessionActive.csv',
                    dtype = {'transferUID': 'object', 'LoginTimes': 'int8'},
                    parse_dates = ['LoginDate'])

df_01.rename(
    inplace = True,
    columns = {
        'LoginDate':'登入日期',
        'transferUID':'玩家帳號',
        'LoginTimes':'登入次數'
    }    
)

# target = pd.read_parquet(base_dir + r'\03_feature data\target.parquet')
# del target['是否流失']


# 02 特徵處理
df_01_copy = df_01.copy()
df_01_copy = df_01_copy[df_01_copy['登入次數'] > 0]
# df_01_copy = df_01.sample(n = 10000)

## 排除掉非目標玩家，排除掉觀察期以外的資料
df_01_copy = pd.merge(target, df_01_copy, on = '玩家帳號', how = 'inner')
df_01_copy = df_01_copy[(obs_start_date <= df_01_copy['登入日期'].dt.date) & (df_01_copy['登入日期'].dt.date <= obs_end_date)]

df_01_copy['周次'] = df_01_copy['登入日期'].dt.isocalendar().week

## 新增特徵: 前第n周量測值，n = 1, 2, 3, 4
df_01_agg = df_01_copy.groupby(['玩家帳號', '周次']).agg({'登入日期':pd.Series.nunique, '登入次數':pd.Series.sum})
df_01_agg.rename(columns = {'登入日期': '登入天數'}, inplace = True)
df_01_agg.reset_index(inplace = True)
df_01_agg['平均日登入次數'] = (df_01_agg['登入天數'] / df_01_agg['登入次數']).round(decimals = 1)

df_01_pivot = df_01_agg.pivot_table(
    index      = ['玩家帳號'], 
    columns    = ['周次'], 
    values     = ['登入天數', '登入次數', '平均日登入次數'], 
    aggfunc    = 'sum', 
    fill_value = 0)

prefix = ['前第' + str(obs_week_number - i) + '周' for i in df_01_pivot.columns.get_level_values(1)]
column_name = df_01_pivot.columns.get_level_values(0)
df_01_pivot.columns = prefix + column_name
df_01_pivot.reset_index(inplace = True)

## 新增特徵: 前1月xxx的周斜率
df_01_slope = calculate_slope(df_01_agg)
df_01_slope.reset_index(inplace = True)
df_01_slope.rename(
    inplace = True,
    columns = {
        'index':'玩家帳號', 
        '登入天數':'前1月登入天數的周斜率',
        '登入次數':'前1月登入次數的周斜率', 
        '平均日登入次數':'前1月平均日登入次數的周斜率'
    }    
)

df_01_slope = standardize_slope(df_01_slope)


## 歸戶至目標玩家
feature_01 = pd.merge(target, df_01_pivot, on = '玩家帳號', how = 'left')
feature_01 = pd.merge(feature_01, df_01_slope, on = '玩家帳號', how = 'left')
feature_01.fillna(value = 0, inplace = True)
del feature_01['是否流失']

# 03 儲存特徵資料
# feature_01.to_parquet(base_dir + r'\03_feature data\feature_01.parquet', index = False)


# 04 呈現資料Shape
# feature_01 = pd.read_parquet(base_dir + r'\03_feature data\feature_01.parquet')

print(f'資料筆數: {feature_01.shape[0]:,}\n欄位數量: {feature_01.shape[1]:}')

feature_01.head()

資料筆數: 75,707
欄位數量: 13
CPU times: total: 11min
Wall time: 11min


,玩家帳號,前第2周平均日登入次數,前第1周平均日登入次數,前第0周平均日登入次數,前第2周登入天數,前第1周登入天數,前第0周登入天數,前第2周登入次數,前第1周登入次數,前第0周登入次數,前1月登入天數的周斜率,前1月登入次數的周斜率,前1月平均日登入次數的周斜率
0,12437148,1,0,0,1,0,0,1,0,0,0,0,0
1,12363509,0.4,0.4,0.5,6,4,1,15,11,2,0,0,0
2,11802253,0,1,0,0,1,0,0,1,0,0,0,0
3,12419390,0.6,0.5,0.3,5,7,1,8,14,3,0,0,0
4,12432207,1,1,1,2,1,1,2,1,1,0,0,0


---
### 02_帳號創建資料
##### accountCreate.csv-> 特徵處理-> feature_02.parquet
[Top](#Table-of-Contents)

In [56]:
%%time
# Wall time: 624 ms

# 01 讀取CSV資料
df_02 = pd.read_csv(base_dir + r'\02_raw data\02_accountCreate\accountCreate.csv',
                    dtype = {'transferUID': 'object', 'Channel': 'object', 'OpenType': 'object'},
                    parse_dates = ['CreateDate'])

df_02_channel = pd.read_csv(base_dir + r'\02_raw data\02_accountCreate\accountCreate_channel.csv',
                            dtype = {'ChannelKey': 'object'})

df_02_optnType = pd.read_csv(base_dir + r'\02_raw data\02_accountCreate\accountCreate_optnType.csv',
                             dtype = {'ThirdPartyType': 'object'})

df_02.rename(
    inplace = True,
    columns = {
        'CreateDate':'建立日期',
        'transferUID':'玩家帳號',
        'Channel':'下載平台',
        'OpenType':'登入平台'
    }    
)

dict_02_channel = df_02_channel.set_index('ChannelKey')['ChannelName'].to_dict()
dict_02_optnType = df_02_optnType.set_index('ThirdPartyType')['ThirdPartyTypeName'].to_dict()

# target = pd.read_parquet(base_dir + r'\03_feature data\target.parquet')
# del target['是否流失']


# 02 特徵處理
df_02_copy = df_02.copy()

df_02_copy['下載平台'] = df_02_copy['下載平台'].map(dict_02_channel)
df_02_copy['登入平台'] = df_02_copy['登入平台'].map(dict_02_optnType)
df_02_copy['是否於觀察期註冊'] = df_02_copy['建立日期'].apply(lambda x: '是' if (obs_start_date <= x.date()) & (x.date() <= obs_end_date) else '否')
df_02_copy['是否於近6個月註冊'] = df_02_copy['建立日期'].apply(lambda x: '是' if (obs_end_date - x.date()).days >= 180 else '否')
del df_02_copy['建立日期']

## 移除重複且不一致的資料
df_02_dup = df_02_copy.groupby(['玩家帳號']).size().reset_index(name='counts')
df_02_dup = df_02_dup[df_02_dup['counts'] > 1]
df_02_dup = pd.merge(df_02_dup, df_02_copy, on = '玩家帳號', how = 'left')
# df_02_dup
list_duplicates = list(df_02_dup['玩家帳號'])
df_02_dedu = df_02_copy[~df_02_copy['玩家帳號'].isin(list_duplicates)]

## 歸戶至目標玩家
feature_02 = pd.merge(target, df_02_dedu, on = '玩家帳號', how = 'left')
feature_02.fillna(value = '沒有紀錄', inplace = True)
del feature_02['是否流失']


# 03 儲存特徵資料
# feature_02.to_parquet(base_dir + r'\03_feature data\feature_02.parquet', index = False)


# 04 呈現資料Shape
# feature_02 = pd.read_parquet(base_dir + r'\03_feature data\feature_02.parquet')

print(f'資料筆數: {feature_02.shape[0]:,}\n欄位數量: {feature_02.shape[1]:}')

feature_02.head()

資料筆數: 75,707
欄位數量: 5
CPU times: total: 125 ms
Wall time: 215 ms


,玩家帳號,下載平台,登入平台,是否於觀察期註冊,是否於近6個月註冊
0,12437148,沒有紀錄,沒有紀錄,沒有紀錄,沒有紀錄
1,12363509,沒有紀錄,沒有紀錄,沒有紀錄,沒有紀錄
2,11802253,沒有紀錄,沒有紀錄,沒有紀錄,沒有紀錄
3,12419390,沒有紀錄,沒有紀錄,沒有紀錄,沒有紀錄
4,12432207,沒有紀錄,沒有紀錄,沒有紀錄,沒有紀錄


---
### 03_遊戲時長資料
##### sessionLength.csv-> 特徵處理-> feature_03.parquet
[Top](#Table-of-Contents)

In [57]:
%%time 
# Wall time: 1min 8s
    
# 01 讀取CSV資料
df_03 = pd.read_csv(base_dir + r'\02_raw data\03_sessionLength\sessionLength.csv',
                    dtype = {'transferUID': 'object', 'GameID': 'object', 'End - Start (Min)': 'int16'},
                    parse_dates = ['StartTime', 'EndTime'])
df_03.rename(
    inplace = True,
    columns = {
        'transferUID':'玩家帳號',
        'GameID':'遊戲代號',
        'StartTime':'開始時間',
        'EndTime':'結束時間',
        'End - Start (Min)':'遊玩時間長度'
    }    
)

df_03_gameId = pd.read_csv(base_dir + r'\02_raw data\03_sessionLength\sessionLength_gameId.csv', dtype = {'GameKey': 'object'})
dict_03_gameName = df_03_gameId.set_index('GameKey')['GameName'].to_dict()
dict_03_gameType = df_03_gameId.set_index('GameKey')['GameType'].to_dict()

# target = pd.read_parquet(base_dir + r'\03_feature data\target.parquet')
# del target['是否流失']


# 02 特徵處理
df_03_copy = df_03.copy()
# df_03_copy = df_03.sample(n = 10000)

## 排除掉非目標玩家，排除掉觀察期以外的資料
df_03_copy = pd.merge(target, df_03_copy, on = '玩家帳號', how = 'inner')
df_03_copy = df_03_copy[(obs_start_date <= df_03_copy['開始時間'].dt.date) & (df_03_copy['開始時間'].dt.date <= obs_end_date)]

df_03_copy['遊戲名稱'] = df_03_copy['遊戲代號'].map(dict_03_gameName)
df_03_copy['遊戲類型'] = df_03_copy['遊戲代號'].map(dict_03_gameType)
df_03_copy['遊戲名稱'].fillna('沒有紀錄', inplace = True)
df_03_copy['遊戲類型'].fillna('沒有紀錄', inplace = True)

df_03_copy['周次'] = df_03_copy['開始時間'].dt.isocalendar().week
df_03_copy['日期'] = df_03_copy['開始時間'].dt.date
del df_03_copy['開始時間']
del df_03_copy['結束時間']

## 新增特徵: 前第n周量測值，n = 1, 2, 3, 4
df_03_agg = df_03_copy.groupby(['玩家帳號', '周次']).agg({'日期':pd.Series.nunique, '遊玩時間長度':pd.Series.sum})
df_03_agg.rename(columns = {'日期': '遊玩天數'}, inplace = True)
df_03_agg.reset_index(inplace = True)
df_03_agg['平均日遊玩時間長度'] = (df_03_agg['遊玩時間長度'] / df_03_agg['遊玩天數']).round(decimals = 1)

df_03_pivot = df_03_agg.pivot_table(
    index      = ['玩家帳號'], 
    columns    = ['周次'], 
    values     = ['遊玩天數', '遊玩時間長度', '平均日遊玩時間長度'], 
    aggfunc    = 'sum', 
    fill_value = 0)

prefix = ['前第' + str(obs_week_number - i) + '周' for i in df_03_pivot.columns.get_level_values(1)]
column_name = df_03_pivot.columns.get_level_values(0)
df_03_pivot.columns = prefix + column_name
df_03_pivot.reset_index(inplace = True)

## 新增特徵: 前1月xxx的周斜率
df_03_slope = calculate_slope(df_03_agg)
df_03_slope.reset_index(inplace = True)
df_03_slope.rename(
    inplace = True,
    columns = {
        'index':'玩家帳號', 
        '遊玩天數':'前1月遊玩天數的周斜率',
        '遊玩時間長度':'前1月遊玩時間長度的周斜率',
        '平均日遊玩時間長度':'前1月平均日遊玩時間長度的周斜率'
    }    
)

df_03_slope = standardize_slope(df_03_slope)


## 歸戶至目標玩家
feature_03 = pd.merge(target, df_03_pivot, on = '玩家帳號', how = 'left')
feature_03 = pd.merge(feature_03, df_03_slope, on = '玩家帳號', how = 'left')
feature_03.fillna(value = 0, inplace = True)
del feature_03['是否流失']


# 03 儲存特徵資料
# feature_03.to_parquet(base_dir + r'\03_feature data\feature_03.parquet', index = False)


# 04 呈現資料Shape
# feature_03 = pd.read_parquet(base_dir + r'\03_feature data\feature_03.parquet')

print(f'資料筆數: {feature_03.shape[0]:,}\n欄位數量: {feature_03.shape[1]:}')

feature_03.head()

資料筆數: 75,707
欄位數量: 13
CPU times: total: 7min 45s
Wall time: 7min 46s


,玩家帳號,前第2周平均日遊玩時間長度,前第1周平均日遊玩時間長度,前第0周平均日遊玩時間長度,前第2周遊玩天數,前第1周遊玩天數,前第0周遊玩天數,前第2周遊玩時間長度,前第1周遊玩時間長度,前第0周遊玩時間長度,前1月遊玩天數的周斜率,前1月遊玩時間長度的周斜率,前1月平均日遊玩時間長度的周斜率
0,12437148,9,0,0,1,0,0,9,0,0,0,0,0
1,12363509,30.7,41.2,4,6,4,1,184,165,4,0,0,0
2,11802253,0,52,0,0,1,0,0,52,0,0,0,0
3,12419390,2.4,3.7,13,5,7,1,12,26,13,0,0,0
4,12432207,11,2,4,1,1,1,11,2,4,0,0,0


---
### 04_遊戲押注紀錄
##### betRecords.csv-> 特徵處理-> feature_04.parquet
[Top](#Table-of-Contents)

In [58]:
%%time
# Wall time: 27.5 s

# 01 讀取CSV資料
df_04 = pd.read_csv(base_dir + r'\02_raw data\04_betRecords\betRecords.csv',
                    parse_dates = ['DateKey'],
                    dtype = {
                        'transferUID': 'object', 
                        'GameID': 'object', 
                        'MachineType': 'object'})
df_04.rename(
    inplace = True,
    columns = {
        'DateKey':'日期',
        'transferUID':'玩家帳號',
        'GameID':'遊戲代號',
        'MachineType':'機台類型',
        'Counts':'押注次數',
        '總贏金幣(不含平台JP)':'總贏金幣',
        '平台JP總贏金幣':'Jackpot金幣'
    }
)

# target = pd.read_parquet(base_dir + r'\03_feature data\target.parquet')
# del target['是否流失']


# 02 特徵處理
df_04_copy = df_04.copy()
# df_04_copy = df_04.sample(n = 10000)

## 排除掉非目標玩家，排除掉觀察期以外的資料
df_04_copy = pd.merge(target, df_04_copy, on = '玩家帳號', how = 'left')
df_04_copy = df_04_copy[(obs_start_date <= df_04_copy['日期'].dt.date) & (df_04_copy['日期'].dt.date <= obs_end_date)]

df_04_copy['周次'] = df_04_copy['日期'].dt.isocalendar().week
del df_04_copy['日期']
del df_04_copy['Jackpot金幣']

## 新增特徵: 前第n周量測值，n = 1, 2, 3, 4
df_04_agg = df_04_copy.groupby(['玩家帳號', '周次']).agg({'押注次數':pd.Series.sum, '總押金幣':pd.Series.sum, '總贏金幣':pd.Series.sum})
df_04_agg['單周報酬率'] = (df_04_agg['總贏金幣'] / df_04_agg['總押金幣']).round(decimals = 4)
df_04_agg['平均押注金額'] = (df_04_agg['總押金幣'] / df_04_agg['押注次數']).round(decimals = 0)
df_04_agg.reset_index(inplace = True)

df_04_pivot = df_04_agg.pivot_table(
    index      = ['玩家帳號'], 
    columns    = ['周次'], 
    values     = ['押注次數', '平均押注金額', '單周報酬率'], 
    aggfunc    = 'sum', 
    fill_value = 0)

prefix = ['前第' + str(obs_week_number - i) + '周' for i in df_04_pivot.columns.get_level_values(1)]
column_name = df_04_pivot.columns.get_level_values(0)
df_04_pivot.columns = prefix + column_name
df_04_pivot.reset_index(inplace = True)

## 新增特徵: 前1月xxx的周斜率
df_04_slope = calculate_slope(df_04_agg)

df_04_slope.reset_index(inplace = True)
df_04_slope.rename(
    inplace = True,
    columns = {
        'index':'玩家帳號', 
        '押注次數':'前1月押注次數的周斜率', 
        '平均押注金額':'前1月平均押注金額的周斜率',
        '單周報酬率':'前1月單周報酬率的周斜率',
        '總押金幣':'前1月總押金幣的周斜率', 
        '總贏金幣':'前1月總贏金幣的周斜率'
    }    
)

df_04_slope = standardize_slope(df_04_slope)

## 歸戶至目標玩家
feature_04 = pd.merge(target, df_04_pivot, on = '玩家帳號', how = 'left')
feature_04 = pd.merge(feature_04, df_04_slope, on = '玩家帳號', how = 'left')
feature_04.fillna(value = 0, inplace = True)
del feature_04['是否流失']


# 03 儲存特徵資料
# feature_04.to_parquet(base_dir + r'\03_feature data\feature_04.parquet', index = False)


# 04 呈現資料Shape
# feature_04 = pd.read_parquet(base_dir + r'\03_feature data\feature_04.parquet')

print(f'資料筆數: {feature_04.shape[0]:,}\n欄位數量: {feature_04.shape[1]:}')

feature_04.head()

資料筆數: 75,707
欄位數量: 15
CPU times: total: 7min 53s
Wall time: 7min 53s


,玩家帳號,前第2周單周報酬率,前第1周單周報酬率,前第0周單周報酬率,前第2周平均押注金額,前第1周平均押注金額,前第0周平均押注金額,前第2周押注次數,前第1周押注次數,前第0周押注次數,前1月押注次數的周斜率,前1月總押金幣的周斜率,前1月總贏金幣的周斜率,前1月單周報酬率的周斜率,前1月平均押注金額的周斜率
0,12437148,2.7141,0,0,15,0,0,49,0,0,0,0,0,0,0
1,12363509,0.9103,0.9761,0.1771,19,48,10,2020,1366,51,0,0,0,0,0
2,11802253,0,0.6692,0,0,65,0,0,9409,0,0,0,0,0,0
3,12419390,0.6076,0.4692,0.704,5,11,14,1234,2624,2223,0,0,0,0,0
4,12432207,0.8617,0.01,0.3313,18,20,10,125,12,47,0,0,0,0,0


---
## 05_金幣進出紀錄
##### coinRecords.csv-> 特徵處理-> feature_05.parquet
[Top](#Table-of-Contents)

In [59]:
%%time
# Wall time: 30.9 s

# 01 讀取CSV資料
df_05 = pd.read_csv(base_dir + r'\02_raw data\05_coinRecords\coinRecords.csv',
                    parse_dates = ['CreateDate'],
                    dtype = {
                        'transferUID': 'object', 
                        'Reason': 'object',
                        'IncreaseCoins': 'float32',
                        'DecreaseCoins': 'float32',
                        'ReasonTimes': 'float32'})
df_05.rename(
    inplace = True, 
    columns = {
        'CreateDate':'日期',
        'transferUID':'玩家帳號',
        'Reason':'事件代號',
        'IncreaseCoins':'增加金幣',
        'DecreaseCoins':'減少金幣',
        'ReasonTimes':'發生次數'
    }
)

df_05_reason = pd.read_csv(base_dir + r'\02_raw data\05_coinRecords\coinRecords_reason.csv', dtype = {'ReasonKey': 'object'})
df_05_reason['Group'] = df_05_reason['Group'].apply(lambda x: re.split(':', x)[1])
dict_05_reason = df_05_reason.set_index('ReasonKey')['Group'].to_dict()

# target = pd.read_parquet(base_dir + r'\03_feature data\target.parquet')
# del target['是否流失']


# 02 特徵處理
df_05_copy = df_05.copy()
# df_05_copy = df_05.sample(n = 10000)

## 排除掉非目標玩家，排除掉觀察期以外的資料
df_05_copy = pd.merge(target, df_05_copy, on = '玩家帳號', how = 'inner')
df_05_copy = df_05_copy[(obs_start_date <= df_05_copy['日期'].dt.date) & (df_05_copy['日期'].dt.date <= obs_end_date)]

df_05_copy['周次'] = df_05_copy['日期'].dt.isocalendar().week
del df_05_copy['日期']

df_05_copy['事件'] = df_05_copy['事件代號'].map(dict_05_reason)
del df_05_copy['事件代號']

df_05_copy.fillna(0)
df_05_copy.replace(np.nan, 0)
df_05_copy['金幣變化'] = (df_05_copy['增加金幣'] - df_05_copy['減少金幣']).round(decimals = 0)
df_05_copy['金幣變化'] = df_05_copy['金幣變化'].astype('float32')

del df_05_copy['增加金幣']
del df_05_copy['減少金幣']

## 新增特徵: 前第n周量測值，n = 1, 2, 3, 4
df_05_agg = df_05_copy.groupby(['玩家帳號', '周次', '事件']).agg({'發生次數':pd.Series.sum, '金幣變化':pd.Series.sum})
df_05_agg.reset_index(inplace = True)

df_05_pivot = df_05_agg.pivot_table(
    index      = ['玩家帳號'], 
    columns    = ['周次', '事件'], 
    values     = ['發生次數', '金幣變化'],
    aggfunc    = 'sum', 
    fill_value = 0)

tmp1 = ['前第' + str(obs_week_number - i) + '周' for i in df_05_pivot.columns.get_level_values(1)]
tmp2 = list(df_05_pivot.columns.get_level_values(2))
tmp3 = list(df_05_pivot.columns.get_level_values(0))
df_05_pivot.columns = [x1 + x2 + '的' + x3 for x1, x2, x3 in zip(tmp1, tmp2, tmp3)]
df_05_pivot.reset_index(inplace = True)

## 新增特徵: 前1月xxx的周斜率
df_05_pivot2 = df_05_agg.pivot_table(
    index      = ['玩家帳號', '周次'], 
    columns    = ['事件'], 
    values     = ['發生次數', '金幣變化'], 
    aggfunc    = 'sum', 
    fill_value = 0)

tmp1 = list(df_05_pivot2.columns.get_level_values(1))
tmp2 = list(df_05_pivot2.columns.get_level_values(0))
df_05_pivot2.columns = [x1 + '的' + x2 for x1, x2 in zip(tmp1, tmp2)]
df_05_pivot2.reset_index(inplace = True)

df_05_slope = calculate_slope(df_05_pivot2)
df_05_slope.reset_index(inplace = True)
df_05_slope.rename(
    inplace = True, 
    columns = {
        'index':'玩家帳號'
    }   
)
df_05_slope = standardize_slope(df_05_slope)

tmp1 = list(df_05_slope.columns)
tmp1.remove('玩家帳號')
tmp2 = ['前1月' + x + '的周斜率' for x in tmp1]
tmp2 = ['玩家帳號'] + tmp2
df_05_slope.columns = tmp2

## 歸戶至目標玩家
feature_05 = pd.merge(target, df_05_pivot, on = '玩家帳號', how = 'left')
feature_05 = pd.merge(feature_05, df_05_slope, on = '玩家帳號', how = 'left')
feature_05.fillna(value = 0, inplace = True)
del feature_05['是否流失']


# 03 儲存特徵資料
# feature_05.to_parquet(base_dir + r'\03_feature data\feature_05.parquet', index = False)


# 04 呈現資料Shape
# feature_05 = pd.read_parquet(base_dir + r'\03_feature data\feature_05.parquet')

print(f'資料筆數: {feature_05.shape[0]:,}\n欄位數量: {feature_05.shape[1]:}')

feature_05.head()

資料筆數: 75,707
欄位數量: 105
CPU times: total: 12min 13s
Wall time: 12min 13s


,玩家帳號,前第2周交易的發生次數,前第2周儲值的發生次數,前第2周其它的發生次數,前第2周平台贈送的發生次數,前第2周平台遊戲的發生次數,前第2周未知的發生次數,前第2周登入贈送的發生次數,前第2周等级贈送的發生次數,前第2周籌碼_常駐贈送的發生次數,前第2周認證贈送的發生次數,前第2周遊戲的發生次數,前第2周運營的發生次數,前第2周道具的發生次數,前第1周交易的發生次數,前第1周儲值的發生次數,前第1周其它的發生次數,前第1周平台贈送的發生次數,前第1周平台遊戲的發生次數,前第1周未知的發生次數,前第1周登入贈送的發生次數,前第1周等级贈送的發生次數,前第1周籌碼_常駐贈送的發生次數,前第1周認證贈送的發生次數,前第1周遊戲的發生次數,前第1周運營的發生次數,前第1周道具的發生次數,前第0周交易的發生次數,前第0周儲值的發生次數,前第0周其它的發生次數,前第0周平台贈送的發生次數,前第0周平台遊戲的發生次數,前第0周未知的發生次數,前第0周登入贈送的發生次數,前第0周等级贈送的發生次數,前第0周籌碼_常駐贈送的發生次數,前第0周認證贈送的發生次數,前第0周遊戲的發生次數,前第0周運營的發生次數,前第0周道具的發生次數,前第2周交易的金幣變化,前第2周儲值的金幣變化,前第2周其它的金幣變化,前第2周平台贈送的金幣變化,前第2周平台遊戲的金幣變化,前第2周未知的金幣變化,前第2周登入贈送的金幣變化,前第2周等级贈送的金幣變化,前第2周籌碼_常駐贈送的金幣變化,前第2周認證贈送的金幣變化,前第2周遊戲的金幣變化,前第2周運營的金幣變化,前第2周道具的金幣變化,前第1周交易的金幣變化,前第1周儲值的金幣變化,前第1周其它的金幣變化,前第1周平台贈送的金幣變化,前第1周平台遊戲的金幣變化,前第1周未知的金幣變化,前第1周登入贈送的金幣變化,前第1周等级贈送的金幣變化,前第1周籌碼_常駐贈送的金幣變化,前第1周認證贈送的金幣變化,前第1周遊戲的金幣變化,前第1周運營的金幣變化,前第1周道具的金幣變化,前第0周交易的金幣變化,前第0周儲值的金幣變化,前第0周其它的金幣變化,前第0周平台贈送的金幣變化,前第0周平台遊戲的金幣變化,前第0周未知的金幣變化,前第0周登入贈送的金幣變化,前第0周等级贈送的金幣變化,前第0周籌碼_常駐贈送的金幣變化,前第0周認證贈送的金幣變化,前第0周遊戲的金幣變化,前第0周運營的金幣變化,前第0周道具的金幣變化,前1月交易的發生次數的周斜率,前1月儲值的發生次數的周斜率,前1月其它的發生次數的周斜率,前1月平台贈送的發生次數的周斜率,前1月平台遊戲的發生次數的周斜率,前1月未知的發生次數的周斜率,前1月登入贈送的發生次數的周斜率,前1月等级贈送的發生次數的周斜率,前1月籌碼_常駐贈送的發生次數的周斜率,前1月認證贈送的發生次數的周斜率,前1月遊戲的發生次數的周斜率,前1月運營的發生次數的周斜率,前1月道具的發生次數的周斜率,前1月交易的金幣變化的周斜率,前1月儲值的金幣變化的周斜率,前1月其它的金幣變化的周斜率,前1月平台贈送的金幣變化的周斜率,前1月平台遊戲的金幣變化的周斜率,前1月未知的金幣變化的周斜率,前1月登入贈送的金幣變化的周斜率,前1月等级贈送的金幣變化的周斜率,前1月籌碼_常駐贈送的金幣變化的周斜率,前1月認證贈送的金幣變化的周斜率,前1月遊戲的金幣變化的周斜率,前1月運營的金幣變化的周斜率,前1月道具的金幣變化的周斜率
0,12437148,0,0,2,0,0,0,1,0,0,0,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,0,0,0,1227,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,12363509,0,0,50,0,0,0,6,9,0,0,1504,0,0,0,0,34,0,0,0,4,5,0,0,1334,0,0,0,0,3,0,0,0,1,1,0,0,51,0,0,0,0,0,0,0,0,1700,1800,0,0,-3469,0,0,0,0,0,0,0,0,400,1000,0,0,-1579,0,0,0,0,0,0,0,0,200,200,0,0,-403,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,11802253,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,12,0,0,0,1,0,0,0,176,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,202200,0,0,0,0,100,0,0,0,-202259,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,12419390,0,0,24,0,0,0,5,8,0,0,20,0,0,0,3,43,0,0,0,7,12,0,0,81,0,0,0,2,14,0,0,0,1,2,0,0,106,0,0,0,0,0,0,0,0,800,1600,0,0,-2546,0,0,0,11700,0,0,0,0,1000,2400,0,0,-15100,0,0,0,8400,0,0,0,0,200,400,0,0,-9000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,12432207,0,0,2,0,0,0,2,1,0,0,119,0,0,0,0,2,0,0,0,1,1,0,0,12,0,0,0,0,2,0,0,0,1,1,0,0,47,0,0,0,0,0,0,0,0,200,150,0,0,-309,0,0,0,0,0,0,0,0,50,150,0,0,-238,0,0,0,0,0,0,0,0,150,150,0,0,-314,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


---
### 06_轉帳交易紀錄
##### txnRecords.csv-> 特徵處理-> feature_06.parquet
[Top](#Table-of-Contents)

In [60]:
%%time
# Wall time: 25.8 s

# 01 讀取CSV資料
df_06 = pd.read_csv(
    base_dir + r'\02_raw data\06_txnRecords\txnRecords.csv',
    parse_dates = ['CreateDate'],
    dtype = {
        'Status': 'object', 
        'TransmitCoin': 'int32', 
        'TransmitFee': 'int32',
        'TtransferUID': 'object',
        'RtransferUID': 'object'
    }
)

df_06.rename(
    inplace = True, 
    columns = {
        'CreateDate':'日期',
        'Status': '交易狀態', 
        'TransmitCoin':'交易金額',
        'TransmitFee':'交易手續費',
        'TtransferUID':'轉出玩家帳號',
        'RtransferUID':'轉入玩家帳號'
    }    
)

df_06_status = pd.read_csv(base_dir + r'\02_raw data\06_txnRecords\txnRecords_status.csv', dtype = {'StatusKey': 'object'})
dict_06_status = df_06_status.set_index('StatusKey')['StatusName'].to_dict()

# target = pd.read_parquet(base_dir + r'\03_feature data\target.parquet')
# del target['是否流失']


# 02 特徵處理
df_06_copy = df_06.copy()
# df_06_copy = df_06.sample(n = 10000)

## 排除掉非目標玩家，排除掉觀察期以外的資料
df_06_copy = df_06_copy[(obs_start_date <= df_06_copy['日期'].dt.date) & (df_06_copy['日期'].dt.date <= obs_end_date)]
df_06_copy = df_06_copy[(df_06_copy['交易狀態'] != '0') & (df_06_copy['交易狀態'] != '1')]

df_06_copy['周次'] = df_06_copy['日期'].dt.isocalendar().week

## 新增特徵: 前第n周量測值，n = 1, 2, 3, 4
df_06_out = df_06_copy.copy()
df_06_out['交易結果'] = df_06_out['交易狀態'].apply(lambda x: '成功轉出' if x == '3' else '失敗轉出')
df_06_out = df_06_out.groupby(['轉出玩家帳號', '周次', '交易結果']).agg({'交易金額':pd.Series.sum, '日期':pd.Series.nunique})
df_06_out.reset_index(inplace = True)
df_06_out.rename(columns = {'轉出玩家帳號': '玩家帳號', '日期': '發生天數'}, inplace = True)
df_06_out['交易金額'] = df_06_out['交易金額'] * (-1)

df_06_in = df_06_copy.copy()
df_06_in['交易結果'] = df_06_in['交易狀態'].apply(lambda x: '成功轉入' if x == '3' else '失敗轉入')
df_06_in = df_06_in.groupby(['轉入玩家帳號', '周次', '交易結果']).agg({'交易金額':pd.Series.sum, '日期':pd.Series.nunique})
df_06_in.reset_index(inplace = True)
df_06_in.rename(columns = {'轉入玩家帳號': '玩家帳號', '日期': '發生天數'}, inplace = True)

df_06_concat = pd.concat([df_06_in, df_06_out])

df_06_pivot = df_06_concat.pivot_table(
    index      = ['玩家帳號'], 
    columns    = ['周次', '交易結果'], 
    values     = ['交易金額', '發生天數'], 
    aggfunc    = 'sum', 
    fill_value = 0)

tmp1 = ['前第' + str(obs_week_number - i) + '周' for i in df_06_pivot.columns.get_level_values(1)]
tmp2 = [x + '的' for x in df_06_pivot.columns.get_level_values(2)]
tmp3 = list(df_06_pivot.columns.get_level_values(0))
df_06_pivot.columns = [x1 + x2 + x3 for x1, x2, x3 in zip(tmp1, tmp2, tmp3)]
df_06_pivot.reset_index(inplace = True)

## 新增特徵: 前1月xxx的周斜率
df_06_pivot2 = df_06_concat.pivot_table(
    index      = ['玩家帳號', '周次'], 
    columns    = ['交易結果'], 
    values     = ['交易金額', '發生天數'], 
    aggfunc    = 'sum', 
    fill_value = 0)

tmp1 = list(df_06_pivot2.columns.get_level_values(1))
tmp2 = list(df_06_pivot2.columns.get_level_values(0))
df_06_pivot2.columns = [x1 + '的' + x2 for x1, x2 in zip(tmp1, tmp2)]
df_06_pivot2.reset_index(inplace = True)

df_06_slope = calculate_slope(df_06_pivot2)
df_06_slope.reset_index(inplace = True)
df_06_slope.rename(
    inplace = True, 
    columns = {
        'index':'玩家帳號'
    }   
)
df_06_slope = standardize_slope(df_06_slope)

tmp1 = list(df_06_slope.columns)
tmp1.remove('玩家帳號')
tmp2 = ['前1月' + x + '的周斜率' for x in tmp1]
tmp2 = ['玩家帳號'] + tmp2
df_06_slope.columns = tmp2

## 歸戶至目標玩家
feature_06 = pd.merge(target, df_06_pivot, on = '玩家帳號', how = 'left')
feature_06 = pd.merge(feature_06, df_06_slope, on = '玩家帳號', how = 'left')
feature_06.fillna(value = 0, inplace = True)
del feature_06['是否流失']


# 03 儲存特徵資料
# feature_06.to_parquet(base_dir + r'\03_feature data\feature_06.parquet', index = False)


# 04 呈現資料Shape
# feature_06 = pd.read_parquet(base_dir + r'\03_feature data\feature_06.parquet')

print(f'資料筆數: {feature_06.shape[0]:,}\n欄位數量: {feature_06.shape[1]:}')

feature_06.head()

資料筆數: 75,707
欄位數量: 33
CPU times: total: 8.31 s
Wall time: 8.4 s


,玩家帳號,前第2周失敗轉入的交易金額,前第2周失敗轉出的交易金額,前第2周成功轉入的交易金額,前第2周成功轉出的交易金額,前第1周失敗轉入的交易金額,前第1周失敗轉出的交易金額,前第1周成功轉入的交易金額,前第1周成功轉出的交易金額,前第0周失敗轉入的交易金額,前第0周失敗轉出的交易金額,前第0周成功轉入的交易金額,前第0周成功轉出的交易金額,前第2周失敗轉入的發生天數,前第2周失敗轉出的發生天數,前第2周成功轉入的發生天數,前第2周成功轉出的發生天數,前第1周失敗轉入的發生天數,前第1周失敗轉出的發生天數,前第1周成功轉入的發生天數,前第1周成功轉出的發生天數,前第0周失敗轉入的發生天數,前第0周失敗轉出的發生天數,前第0周成功轉入的發生天數,前第0周成功轉出的發生天數,前1月失敗轉入的交易金額的周斜率,前1月失敗轉出的交易金額的周斜率,前1月成功轉入的交易金額的周斜率,前1月成功轉出的交易金額的周斜率,前1月失敗轉入的發生天數的周斜率,前1月失敗轉出的發生天數的周斜率,前1月成功轉入的發生天數的周斜率,前1月成功轉出的發生天數的周斜率
0,12437148,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,12363509,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,11802253,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,12419390,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,12432207,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


---
### 07_儲值紀錄
##### depositRecords.csv-> 特徵處理-> feature_07.parquet
[Top](#Table-of-Contents)

In [61]:
%%time
# Wall time: 2.11 s

# 01 讀取CSV資料
df_07 = pd.read_csv(
    base_dir + r'\02_raw data\07_depositRecords\depositRecords.csv',
    parse_dates = ['CreateDate'],
    dtype = {
        'transferUID': 'object'
    }
)

df_07.rename(
    columns = {
        'CreateDate': '日期', 
        'transferUID': '玩家帳號', 
        'BuyNumber': '儲值金額', 
        'CoinAfter': '儲值前餘額',
        'CoinBefore': '儲值後餘額'
    },
    inplace = True
)

# target = pd.read_parquet(base_dir + r'\03_feature data\target.parquet')
# del target['是否流失']


# 02 特徵處理
df_07_copy = df_07.copy()
# df_07_copy = df_07.sample(n = 10000)

## 排除掉非目標玩家，排除掉觀察期以外的資料
df_07_copy = pd.merge(target, df_07_copy, on = '玩家帳號', how = 'inner')
df_07_copy = df_07_copy[(obs_start_date <= df_07_copy['日期'].dt.date) & (df_07_copy['日期'].dt.date <= obs_end_date)]

df_07_copy['周次'] = df_07_copy['日期'].dt.isocalendar().week
del df_07_copy['日期']

## 新增特徵: 前第n周量測值，n = 1, 2, 3, 4
df_07_agg = df_07_copy.groupby(['玩家帳號', '周次']).agg({'儲值金額':pd.Series.sum, '儲值前餘額':[np.min, np.mean]})
df_07_agg.reset_index(inplace = True)
df_07_agg.columns = df_07_agg.columns.get_level_values(0) + df_07_agg.columns.get_level_values(1)
df_07_agg.rename(
    inplace = True,
    columns = {
        '儲值金額sum': '儲值金額', 
        '儲值前餘額amin': '最小餘額',
        '儲值前餘額mean':'平均餘額'
    }
)

df_07_pivot = df_07_agg.pivot_table(
    index      = ['玩家帳號'], 
    columns    = ['周次'], 
    values     = ['儲值金額', '最小餘額', '平均餘額'], 
    aggfunc    = 'sum', 
    fill_value = 0)

tmp1 = ['前第' + str(obs_week_number - i) + '周' for i in df_07_pivot.columns.get_level_values(1)]
tmp2 = list(df_07_pivot.columns.get_level_values(0))
df_07_pivot.columns = [x1 + x2 for x1, x2 in zip(tmp1, tmp2)]
df_07_pivot.reset_index(inplace = True)

## 新增特徵: 前1月xxx的周斜率
df_07_slope = calculate_slope(df_07_agg)
df_07_slope.reset_index(inplace = True)
df_07_slope.rename(inplace = True, columns = {'index':'玩家帳號'})
df_07_slope = standardize_slope(df_07_slope)

tmp1 = list(df_07_slope.columns)
tmp1.remove('玩家帳號')
tmp2 = ['前1月' + x + '的周斜率' for x in tmp1]
tmp2 = ['玩家帳號'] + tmp2
df_07_slope.columns = tmp2

## 歸戶至目標玩家
feature_07 = pd.merge(target, df_07_pivot, on = '玩家帳號', how = 'left')
feature_07 = pd.merge(feature_07, df_07_slope, on = '玩家帳號', how = 'left')
feature_07.fillna(value = 0, inplace = True)
del feature_07['是否流失']


# 03 儲存特徵資料
# feature_07.to_parquet(base_dir + r'\03_feature data\feature_07.parquet', index = False)


# 04 呈現資料Shape
# feature_07 = pd.read_parquet(base_dir + r'\03_feature data\feature_07.parquet')

print(f'資料筆數: {feature_07.shape[0]:,}\n欄位數量: {feature_07.shape[1]:}')

feature_07.head()

資料筆數: 75,707
欄位數量: 13
CPU times: total: 13.9 s
Wall time: 14 s


,玩家帳號,前第2周儲值金額,前第1周儲值金額,前第0周儲值金額,前第2周平均餘額,前第1周平均餘額,前第0周平均餘額,前第2周最小餘額,前第1周最小餘額,前第0周最小餘額,前1月儲值金額的周斜率,前1月最小餘額的周斜率,前1月平均餘額的周斜率
0,12437148,0,0,0,0,0,0,0,0,0,0,0,0
1,12363509,0,0,0,0,0,0,0,0,0,0,0,0
2,11802253,0,1950,0,0,30319.2,0,0,15102,0,0,0,0
3,12419390,0,103,70,0,5151.9,7000,0,3303.8,7000,0,0,0
4,12432207,0,0,0,0,0,0,0,0,0,0,0,0


---
### 08_儲值退費紀錄
##### refundRecords.csv-> 特徵處理-> feature_08.parquet
[Top](#Table-of-Contents)

In [62]:
%%time
# Wall time: 117 ms

# 01 讀取CSV資料
df_08 = pd.read_csv(
    base_dir + r'\02_raw data\08_refundRecords\refundRecords.csv', 
    parse_dates = ['DoneTime'], 
    dtype = {'transferUID': 'object'}
)

del df_08['EventTime']
del df_08['VoidedTime']

df_08.rename(
    inplace = True,
    columns = {
        'DoneTime': '日期', 
        'transferUID': '玩家帳號',
        'OrderID':'訂單編號',
        'BuyNumber': '退費金額', 
        'Distributor': '儲值管道代碼'
    }
)

df_08_distributor = pd.read_csv(base_dir + r'\02_raw data\08_refundRecords\refundRecords_distributor.csv', 
                                dtype = {'DistributorKey': 'object'})

dict_08_distributor = df_08_distributor.set_index('DistributorKey')['DistributorName'].to_dict()

# target = pd.read_parquet(base_dir + r'\03_feature data\target.parquet')
# del target['是否流失']


# 02 特徵處理
df_08_copy = df_08.copy()

## 排除掉非目標玩家，排除掉觀察期以外的資料
df_08_copy = pd.merge(target, df_08_copy, on = '玩家帳號', how = 'inner')
df_08_copy = df_08_copy[(obs_start_date <= df_08_copy['日期'].dt.date) & (df_08_copy['日期'].dt.date <= obs_end_date)]

df_08_copy['周次'] = df_08_copy['日期'].dt.isocalendar().week
del df_08_copy['日期']

## 新增特徵: 前第n周量測值，n = 1, 2, 3, 4
df_08_agg = df_08_copy.groupby(['玩家帳號', '周次']).agg({'退費金額':pd.Series.sum, '訂單編號':pd.Series.nunique})
df_08_agg.reset_index(inplace = True)
df_08_agg.rename(
    inplace = True,
    columns = {
        '訂單編號': '退費次數'
    }
)

df_08_pivot = df_08_agg.pivot_table(
    index      = ['玩家帳號'], 
    columns    = ['周次'], 
    values     = ['退費金額', '退費次數'], 
    aggfunc    = 'sum', 
    fill_value = 0)

tmp1 = ['前第' + str(obs_week_number - i) + '周' for i in df_08_pivot.columns.get_level_values(1)]
tmp2 = list(df_08_pivot.columns.get_level_values(0))
df_08_pivot.columns = [x1 + x2 for x1, x2 in zip(tmp1, tmp2)]
df_08_pivot.reset_index(inplace = True)

## 新增特徵: 前1月是否退費
df_08_agg2 = df_08_agg.groupby(['玩家帳號']).agg({'退費次數':pd.Series.sum})
df_08_merge = pd.merge(target, df_08_agg2, on = '玩家帳號', how = 'left', indicator = True)
df_08_merge['前1月是否退費'] = df_08_merge['_merge'].apply(lambda x: '是' if x == 'left_only' else '否')
del df_08_merge['_merge']
df_08_refund = df_08_merge[['玩家帳號', '前1月是否退費']]

## 歸戶至目標玩家
feature_08 = pd.merge(df_08_refund, df_08_pivot, on = '玩家帳號', how = 'left')
feature_08.fillna(value = 0, inplace = True)
#del feature_08['是否流失']


# 03 儲存特徵資料
# feature_08.to_parquet(base_dir + r'\03_feature data\feature_08.parquet', index = False)


# 04 呈現資料Shape
# feature_08 = pd.read_parquet(base_dir + r'\03_feature data\feature_08.parquet')

print(f'資料筆數: {feature_08.shape[0]:,}\n欄位數量: {feature_08.shape[1]:}')

feature_08.head()

資料筆數: 75,707
欄位數量: 6
CPU times: total: 156 ms
Wall time: 211 ms


,玩家帳號,前1月是否退費,前第2周退費次數,前第1周退費次數,前第2周退費金額,前第1周退費金額
0,12437148,是,0,0,0,0
1,12363509,是,0,0,0,0
2,11802253,是,0,0,0,0
3,12419390,是,0,0,0,0
4,12432207,是,0,0,0,0


---
### 合併所有特徵
[Top](#Table-of-Contents)

In [63]:
%%time

feature_sets = [feature_01, feature_02, feature_03, feature_04, feature_05, feature_06, feature_07, feature_08]
data = target.copy()

# 01 合併所有特徵
for f in feature_sets:
    data = pd.merge(data, f, on = '玩家帳號', how = 'left')

data.head()
# 03 呈現資料Shape
print(f'資料筆數: {data.shape[0]:,}\n欄位數量: {data.shape[1]:}')
data.head(100)

資料筆數: 75,707
欄位數量: 197
CPU times: total: 703 ms
Wall time: 704 ms


,玩家帳號,是否流失,前第2周平均日登入次數,前第1周平均日登入次數,前第0周平均日登入次數,前第2周登入天數,前第1周登入天數,前第0周登入天數,前第2周登入次數,前第1周登入次數,前第0周登入次數,前1月登入天數的周斜率,前1月登入次數的周斜率,前1月平均日登入次數的周斜率,下載平台,登入平台,是否於觀察期註冊,是否於近6個月註冊,前第2周平均日遊玩時間長度,前第1周平均日遊玩時間長度,前第0周平均日遊玩時間長度,前第2周遊玩天數,前第1周遊玩天數,前第0周遊玩天數,前第2周遊玩時間長度,前第1周遊玩時間長度,前第0周遊玩時間長度,前1月遊玩天數的周斜率,前1月遊玩時間長度的周斜率,前1月平均日遊玩時間長度的周斜率,前第2周單周報酬率,前第1周單周報酬率,前第0周單周報酬率,前第2周平均押注金額,前第1周平均押注金額,前第0周平均押注金額,前第2周押注次數,前第1周押注次數,前第0周押注次數,前1月押注次數的周斜率,前1月總押金幣的周斜率,前1月總贏金幣的周斜率,前1月單周報酬率的周斜率,前1月平均押注金額的周斜率,前第2周交易的發生次數,前第2周儲值的發生次數,前第2周其它的發生次數,前第2周平台贈送的發生次數,前第2周平台遊戲的發生次數,前第2周未知的發生次數,前第2周登入贈送的發生次數,前第2周等级贈送的發生次數,前第2周籌碼_常駐贈送的發生次數,前第2周認證贈送的發生次數,前第2周遊戲的發生次數,前第2周運營的發生次數,前第2周道具的發生次數,前第1周交易的發生次數,前第1周儲值的發生次數,前第1周其它的發生次數,前第1周平台贈送的發生次數,前第1周平台遊戲的發生次數,前第1周未知的發生次數,前第1周登入贈送的發生次數,前第1周等级贈送的發生次數,前第1周籌碼_常駐贈送的發生次數,前第1周認證贈送的發生次數,前第1周遊戲的發生次數,前第1周運營的發生次數,前第1周道具的發生次數,前第0周交易的發生次數,前第0周儲值的發生次數,前第0周其它的發生次數,前第0周平台贈送的發生次數,前第0周平台遊戲的發生次數,前第0周未知的發生次數,前第0周登入贈送的發生次數,前第0周等级贈送的發生次數,前第0周籌碼_常駐贈送的發生次數,前第0周認證贈送的發生次數,前第0周遊戲的發生次數,前第0周運營的發生次數,前第0周道具的發生次數,前第2周交易的金幣變化,前第2周儲值的金幣變化,前第2周其它的金幣變化,前第2周平台贈送的金幣變化,前第2周平台遊戲的金幣變化,前第2周未知的金幣變化,前第2周登入贈送的金幣變化,前第2周等级贈送的金幣變化,前第2周籌碼_常駐贈送的金幣變化,前第2周認證贈送的金幣變化,前第2周遊戲的金幣變化,前第2周運營的金幣變化,前第2周道具的金幣變化,前第1周交易的金幣變化,前第1周儲值的金幣變化,前第1周其它的金幣變化,前第1周平台贈送的金幣變化,前第1周平台遊戲的金幣變化,前第1周未知的金幣變化,前第1周登入贈送的金幣變化,前第1周等级贈送的金幣變化,前第1周籌碼_常駐贈送的金幣變化,前第1周認證贈送的金幣變化,前第1周遊戲的金幣變化,前第1周運營的金幣變化,前第1周道具的金幣變化,前第0周交易的金幣變化,前第0周儲值的金幣變化,前第0周其它的金幣變化,前第0周平台贈送的金幣變化,前第0周平台遊戲的金幣變化,前第0周未知的金幣變化,前第0周登入贈送的金幣變化,前第0周等级贈送的金幣變化,前第0周籌碼_常駐贈送的金幣變化,前第0周認證贈送的金幣變化,前第0周遊戲的金幣變化,前第0周運營的金幣變化,前第0周道具的金幣變化,前1月交易的發生次數的周斜率,前1月儲值的發生次數的周斜率,前1月其它的發生次數的周斜率,前1月平台贈送的發生次數的周斜率,前1月平台遊戲的發生次數的周斜率,前1月未知的發生次數的周斜率,前1月登入贈送的發生次數的周斜率,前1月等级贈送的發生次數的周斜率,前1月籌碼_常駐贈送的發生次數的周斜率,前1月認證贈送的發生次數的周斜率,前1月遊戲的發生次數的周斜率,前1月運營的發生次數的周斜率,前1月道具的發生次數的周斜率,前1月交易的金幣變化的周斜率,前1月儲值的金幣變化的周斜率,前1月其它的金幣變化的周斜率,前1月平台贈送的金幣變化的周斜率,前1月平台遊戲的金幣變化的周斜率,前1月未知的金幣變化的周斜率,前1月登入贈送的金幣變化的周斜率,前1月等级贈送的金幣變化的周斜率,前1月籌碼_常駐贈送的金幣變化的周斜率,前1月認證贈送的金幣變化的周斜率,前1月遊戲的金幣變化的周斜率,前1月運營的金幣變化的周斜率,前1月道具的金幣變化的周斜率,前第2周失敗轉入的交易金額,前第2周失敗轉出的交易金額,前第2周成功轉入的交易金額,前第2周成功轉出的交易金額,前第1周失敗轉入的交易金額,前第1周失敗轉出的交易金額,前第1周成功轉入的交易金額,前第1周成功轉出的交易金額,前第0周失敗轉入的交易金額,前第0周失敗轉出的交易金額,前第0周成功轉入的交易金額,前第0周成功轉出的交易金額,前第2周失敗轉入的發生天數,前第2周失敗轉出的發生天數,前第2周成功轉入的發生天數,前第2周成功轉出的發生天數,前第1周失敗轉入的發生天數,前第1周失敗轉出的發生天數,前第1周成功轉入的發生天數,前第1周成功轉出的發生天數,前第0周失敗轉入的發生天數,前第0周失敗轉出的發生天數,前第0周成功轉入的發生天數,前第0周成功轉出的發生天數,前1月失敗轉入的交易金額的周斜率,前1月失敗轉出的交易金額的周斜率,前1月成功轉入的交易金額的周斜率,前1月成功轉出的交易金額的周斜率,前1月失敗轉入的發生天數的周斜率,前1月失敗轉出的發生天數的周斜率,前1月成功轉入的發生天數的周斜率,前1月成功轉出的發生天數的周斜率,前第2周儲值金額,前第1周儲值金額,前第0周儲值金額,前第2周平均餘額,前第1周平均餘額,前第0周平均餘額,前第2周最小餘額,前第1周最小餘額,前第0周最小餘額,前1月儲值金額的周斜率,前1月最小餘額的周斜率,前1月平均餘額的周斜率,前1月是否退費,前第2周退費次數,前第1周退費次數,前第2周退費金額,前第1周退費金額
0,12437148,是,1,0,0,1,0,0,1,0,0,0,0,0,沒有紀錄,沒有紀錄,沒有紀錄,沒有紀錄,9,0,0,1,0,0,9,0,0,0,0,0,2.7141,0,0,15,0,0,49,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,0,0,0,1227,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,是,0,0,0,0
1,12363509,是,0.4,0.4,0.5,6,4,1,15,11,2,0,0,0,沒有紀錄,沒有紀錄,沒有紀錄,沒有紀錄,30.7,41.2,4,6,4,1,184,165,4,0,0,0,0.9103,0.9761,0.1771,19,48,10,2020,1366,51,0,0,0,0,0,0,0,50,0,0,0,6,9,0,0,1504,0,0,0,0,34,0,0,0,4,5,0,0,1334,0,0,0,0,3,0,0,0,1,1,0,0,51,0,0,0,0,0,0,0,0,1700,1800,0,0,-3469,0,0,0,0,0,0,0,0,400,1000,0,0,-1579,0,0,0,0,0,0,0,0,200,200,0,0,-403,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,是,0,0,0,0
2,11802253,是,0,1,0,0,1,0,0,1,0,0,0,0,沒有紀錄,沒有紀錄,沒有紀錄,沒有紀錄,0,52,0,0,1,0,0,52,0,0,0,0,0,0.6692,0,0,65,0,0,9409,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,12,0,0,0,1,0,0,0,176,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,202200,0,0,0,0,100,0,0,0,-202259,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1950,0,0,30319.2,0,0,15102,0,0,0,0,是,0,0,0,0
3,12419390,是,0.6,0.5,0.3,5,7,1,8,14,3,0,0,0,沒有紀錄,沒有紀錄,沒有紀錄,沒有紀錄,2.4,3.7,13,5,7,

In [64]:
columns_list = list(data.columns)
print("合併後的欄位列表：", columns_list)


合併後的欄位列表： ['玩家帳號', '是否流失', '前第2周平均日登入次數', '前第1周平均日登入次數', '前第0周平均日登入次數', '前第2周登入天數', '前第1周登入天數', '前第0周登入天數', '前第2周登入次數', '前第1周登入次數', '前第0周登入次數', '前1月登入天數的周斜率', '前1月登入次數的周斜率', '前1月平均日登入次數的周斜率', '下載平台', '登入平台', '是否於觀察期註冊', '是否於近6個月註冊', '前第2周平均日遊玩時間長度', '前第1周平均日遊玩時間長度', '前第0周平均日遊玩時間長度', '前第2周遊玩天數', '前第1周遊玩天數', '前第0周遊玩天數', '前第2周遊玩時間長度', '前第1周遊玩時間長度', '前第0周遊玩時間長度', '前1月遊玩天數的周斜率', '前1月遊玩時間長度的周斜率', '前1月平均日遊玩時間長度的周斜率', '前第2周單周報酬率', '前第1周單周報酬率', '前第0周單周報酬率', '前第2周平均押注金額', '前第1周平均押注金額', '前第0周平均押注金額', '前第2周押注次數', '前第1周押注次數', '前第0周押注次數', '前1月押注次數的周斜率', '前1月總押金幣的周斜率', '前1月總贏金幣的周斜率', '前1月單周報酬率的周斜率', '前1月平均押注金額的周斜率', '前第2周交易的發生次數', '前第2周儲值的發生次數', '前第2周其它的發生次數', '前第2周平台贈送的發生次數', '前第2周平台遊戲的發生次數', '前第2周未知的發生次數', '前第2周登入贈送的發生次數', '前第2周等级贈送的發生次數', '前第2周籌碼_常駐贈送的發生次數', '前第2周認證贈送的發生次數', '前第2周遊戲的發生次數', '前第2周運營的發生次數', '前第2周道具的發生次數', '前第1周交易的發生次數', '前第1周儲值的發生次數', '前第1周其它的發生次數', '前第1周平台贈送的發生次數', '前第1周平台遊戲的發生次數', '前第1周未知的發生次數', '前第1周登入贈送的發生次數', '前第1周等级贈送的發生次數', '前第1周籌碼_常駐贈送的發生次數', '前第1周認證贈送的發生次數', '前第1周遊戲的發生次數', '

---
# Part2 玩家流失模型建置
### 1. 資料處理
[Top](#Table-of-Contents)

In [68]:
# New try

# 01 針對類別資料作one-hot encoding (即生成dummy variable)
# data = pd.read_parquet(base_dir + r'\03_feature data\data.parquet')
data.replace([np.inf, -np.inf], np.nan, inplace = True)
data = data.fillna(0)
# del data['玩家帳號']
# 將資料集分成兩部分，這裡取前一半的資料
data_half = data.iloc[:len(data)//2, :]

data_half_2 = data.iloc[len(data)//2:, :]
#決定要用哪一半轉成dummy
data_dum = pd.get_dummies(data_half_2) #這邊記得要選哪一半
#del data_dum['是否流失_否']
data_dum.rename(inplace = True, columns = {'是否流失_是':'是否流失'})

## 產生X、Y資料集(模型建立拿取features_selected)
features_selected = ['前第1周儲值的發生次數', '前第0周運營的發生次數', '前第1周平均押注金額', '前第1周等级贈送的金幣變化', '前第1周交易的發生次數', '前第2周退費金額', '前第1周退費金額', '前第0周登入次數', '前第0周未知的發生次數', '登入平台_AppleLogin', '前第1周登入贈送的金幣變化', '前第0周等级贈送的金幣變化', '前第1周運營的發生次數', '前第2周平台贈送的發生次數', '前第2周遊戲的金幣變化', '前第2周籌碼_常駐贈送的金幣變化', '前第2周單周報酬率', '前第0周單周報酬率', '前第1周成功轉入的發生天數', '前第2周道具的發生次數', '前第2周登入贈送的金幣變化', '登入平台_phone_number', '前第1周押注次數', '前第0周平台贈送的發生次數', '前第1周平均日登入次數', '前第1周籌碼_常駐贈送的發生次數', '前第2周平均日遊玩時間長度', '前第2周等级贈送的金幣變化', '登入平台_IGS', '前第0周認證贈送的發生次數', '前第2周平均日登入次數', '登入平台_Line', '前第2周未知的發生次數', '前第0周平均日遊玩時間長度', '前第1周遊戲的金幣變化', '前第1周登入天數', '前第0周平台遊戲的發生次數', '前第0周儲值的發生次數', '是否於觀察期註冊_是', '前第1周認證贈送的發生次數', '前第1周平台贈送的金幣變化', '前第0周道具的發生次數', '前第1周單周報酬率', '前第2周其它的發生次數', '前第1周未知的發生次數', '前第0周平均日登入次數']
#使用時此段先註解
# c = Counter(data_dum['是否流失'])
# print(f'玩家人數: {len(data_dum["是否流失"]):,}')
# print(f'流失人數: {c.get(1):,}')
# print(f'流失比例: {c.get(0)/len(data["是否流失"]):.2%}')


# # 02 維度縮減 dimension reduction
# ## 移除高共線性的數值變數
# snc = SelectNonCollinear()
# snc.fit(X.to_numpy(), Y.to_numpy())
# snc_selected = snc.get_support()
# X_nonCollinear = X.loc[:, snc_selected]
# snc_selected = X_nonCollinear.columns.to_list()
# # sns.heatmap(X_nonCollinear.corr().abs(), annot = True)

# ## Regression-based 特徵選取
# lsvc = LinearSVC(C = 0.01, penalty = "l1", dual = False)
# lsvc.fit(X_nonCollinear, Y)
# lsvc_model = SelectFromModel(lsvc, prefit = True)
# lsvc_selected = lsvc_model.get_support()

# X_lsvc = X_nonCollinear.loc[:, lsvc_selected]
# # list(X_lsvc.columns)

# ## Tree-based 特徵選取
# etc = ExtraTreesClassifier(n_estimators = 50)
# etc = etc.fit(X_nonCollinear, Y)
# etc_model = SelectFromModel(etc, prefit = True)
# etc_selected = etc_model.get_support()

# X_etc = X_nonCollinear.loc[:, etc_selected]
# # list(X_etc.columns)

# ## 取L1-based、tree-based特徵的聯集
# features_selected = set(X_lsvc.columns)
# features_selected.update(set(X_etc.columns))

# X_selected = X[list(features_selected)]
X_selected = data_dum[features_selected]
Y = data_dum['是否流失']

# 03 將資料分割成訓練集 (80%)、測試集 (20%)
# X_train, X_test, Y_train, Y_test = train_test_split(X_selected, Y, test_size = 0.20, random_state = 2022)

# 04 將資料從pandas dataframe格式轉成XGboost模型專用的DMatrix格式，可以提升XGBoost演算效率
# X_train_dm = xgb.DMatrix(X_train, label = Y_train, enable_categorical = True)
# X_test_dm = xgb.DMatrix(X_test, label = Y_test, enable_categorical = True)
X_dm = xgb.DMatrix(X_selected, label = Y, enable_categorical = True)

### 2. 模型使用
[Top](#Table-of-Contents)

In [69]:
# 01 讀取訓練好的XGBoost模型
model_xgb = xgb.Booster()
model_xgb.load_model('./model_xgb.json')
model_xgb.attributes()

# 02 使用模型作預測
Y_pred = model_xgb.predict(X_dm)

# # 03 計算模型準確度 (注意事項：本案已經知道實際知道這些玩家是否流失，所以才能計算得出來)
# def plot_confusion_matrix(data):
#     plt.figure(1, figsize = (5, 5)) 
#     plt.title("Confusion Matrix")
    
#     sns.color_palette(palette = "mako")
    
#     ax = sns.heatmap(data, annot=True, fmt='.2%', cmap="YlGnBu", cbar_kws={'label': 'Scale'})
 
#     ax.set(ylabel="Ground Truth", xlabel="Predicted Label")
 
#     plt.show() 
    
# cfMatrix = confusion_matrix(Y, Y_pred)
# plot_confusion_matrix(data = cfMatrix/ cfMatrix.sum(axis = 1))

# accuracy = accuracy_score(Y, Y_pred)
# f1 = f1_score(Y, Y_pred)
# recall = recall_score(Y, Y_pred)
# precision = precision_score(Y, Y_pred)

# print(f'模型準確率: {accuracy:.2%}', end = '\n')
# print(f'模型F1分數: {f1:.2%}', end = '\n')
# print(f'流失玩家中，模型可以抓到的比例: {recall:.2%}', end = '\n')
# print(f'模型認為的流失玩家中，真正為流失的比例: {precision:.2%}', end = '\n\n')

#target_half= target.iloc[:len(target), :]
target_half = target.iloc[:len(target)//2, :]
target_half_2 = target.iloc[len(target)//2:, :]

# 04 輸出流失玩家清單
churn_customer_list = pd.DataFrame({'玩家帳號': target_half_2['玩家帳號'], '是否流失_預測值': Y_pred})
churn_customer_list.to_csv('./玩家流失名單2.csv', index = False, encoding = 'utf_8_sig')
churn_customer_list.head(20)

,玩家帳號,是否流失_預測值
37853,10813886,1
37854,11703181,0
37855,12393211,0
37856,12393268,0
37857,11739373,0
37858,10191602,1
37859,10741394,0
37860,12449925,1
37861,12427411,1
37862,12405380,0


---
<div style="text-align: right"> 品俞 202410</div>